# Main file

## 00. Imports, constants and constants' functions

In [11]:
import os
import sys
import sqlite3
import pandas as pd
import time
import copy
import numpy as np
import pickle
import datetime
import re
import pulp
import tqdm
import datetime
import pathlib
import importlib

sys.path.append('') # # Define your own folder
import Send_email # my script
import Results_analysis # my script

from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, BayesianRidge, ElasticNet, RidgeCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import r2_score

os.chdir('') # Define your own folder
connection = sqlite3.connect('') # Define your own folder
date_extension = str(datetime.date.today())
review_dictionary = {} # initialize dictionary to collect relevant statistics for a final report

## 0. Constants and basic data 

In [12]:
class Constants:  
    AVAILABLE_SEASONS = ['2016-17', '2017-18', '2018-19', '2019-20']
    
    BASIC_BOXSCORE_ITEMS = ['GAME_ID', 'TEAM_ID','TEAM_ABBREVIATION','PLAYER_ID', 'MIN', 'PTS',
                        'REB', 'AST', 'STL', 'BLK', 'TO', 'FGM', 'FGA', 'FG_PCT']
    
    BASIC_STATS = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'TO', 'FGM', 'FGA', 'FG_PCT', 'DF_PTS']
    
    ADVANCED_STATS = ['AST_RATIO','AST_TOV','DEF_RATING', 'DREB_PCT', 'EFG_PCT', 'E_DEF_RATING', 'E_NET_RATING',
                     'E_OFF_RATING', 'E_PACE', 'E_TM_TOV_PCT', 'NET_RATING', 'OFF_RATING',
                    'OREB_PCT', 'PACE', 'PIE', 'REB_PCT', 'TM_TOV_PCT', 'TS_PCT']
    
    POSITIONAL_LIMITS = {'PG' : 3, 'SG': 3, 'SF': 3, 'PF': 3, 'C':3, 'TOTAL':7}
        
lag1, lag2, lag3 = 1, 5, 15 # Lags to be used for rolling averages.

In [13]:
def df_points(df): # Define formula to be used for optimization goal
    DF_PTS = df.PTS + 1.25 * df.REB + 1.5 * df.AST + 2 * df.STL + 2 * df.BLK - 0.5 * df.TO
    return DF_PTS


def player_list(): # Create a list of nba players with their IDs
    players2019 = pd.read_sql_query("SELECT * FROM Player_list", connection).drop(
        columns = ['PLAYER_POSITIONS'], axis = 1).reset_index(drop=True)
    if datetime.date.today().day == 1 and input('Do you want to update player list? 1/0 ') == 1:
        players2019 = players.get_active_players()
        players2019 = pd.DataFrame(players2019).drop(columns='is_active')     
        players2019.to_sql('Player_list', connection, if_exists='replace')    
    review_dictionary.update({'total number of players': players2019.shape[0]})
    return players2019


def get_games20162020(): # Get the list of games played over last seasons   
    games = leaguegamelog.LeagueGameLog(season='2019-20')
    games = games.league_game_log.get_data_frame().loc[:,['GAME_ID', 'GAME_DATE', 'TEAM_ID', 'MATCHUP']]
    unique_games = games.drop_duplicates(subset = 'GAME_ID')  
    return unique_games


def get_game_dates(): # Obtain game dates and the use it to insert in case missing
    game_dates = leaguegamelog.LeagueGameLog(season='2019-20')
    game_dates = game_dates.league_game_log.get_data_frame().loc[:,['GAME_ID', 'GAME_DATE']]
    game_dates = game_dates.drop_duplicates(subset = ['GAME_ID'])
    game_dates['GAME_ID'] = game_dates['GAME_ID'].astype(int)
    return game_dates
    
    
def switching_columns(lag1, lag2, lag3): # From advanced stats make the list of columns of rolling averages
    switching_columns_1 = []
    switching_columns_2 = []
    for stat in Constants.ADVANCED_STATS:
        for lag in [lag1, lag2, lag3]:
            item = ['Last{x}_{y}'.format(x=lag, y=stat)]
            switching_columns_1.append(item)
    switching_columns_2 = copy.deepcopy(switching_columns_1)
    switching_columns_1.append(['GAME_ID', 'TEAM_ID'])
    # Flatten nested lists (lists of lists) that were created, can't iterate if not flat.
    switching_columns_1 = [item for sublist in switching_columns_1 for item in sublist]
    switching_columns_2 = [item for sublist in switching_columns_2 for item in sublist]
    return switching_columns_1, switching_columns_2


def rolling_average(df, window, shift): # Function to calculate rolling averages
    return df.rolling(min_periods=window, window=window).mean().shift(shift)


def drop_y(df): # After merging two dataframes, unnecessasry columns with _y are added 
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(columns = to_drop, axis = 1, inplace = True)
    for col in df:
        if col.endswith('_x'):
            df.rename(columns = {col:col.rstrip('_x')}, inplace = True)

## Download basic and advanced boxscores from scratch or update with missing ones

In [14]:
def basic_boxscores(GamesIDs, seconds): # Download basic boxscores for chosen IDs
    pd.options.mode.chained_assignment = None
    BoxScore_df = pd.DataFrame()
    for ID in tqdm.tqdm(GamesIDs['GAME_ID']):
        time.sleep(seconds)
        BoxScore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=ID)
        BoxScore = BoxScore.player_stats.get_data_frame()
        BoxScore = BoxScore.loc[:, Constants.BASIC_BOXSCORE_ITEMS] # filters only the chosen boxscore items.
        BoxScore['DF_PTS'] = df_points(BoxScore) # Function calculates daily fantasy points.
        BoxScore_df = BoxScore_df.append(BoxScore) 
        BoxScore_df = BoxScore_df.drop_duplicates(subset=['GAME_ID','PLAYER_ID'], keep='first')
    BoxScore_df = BoxScore_df.replace("", np.nan, regex=True)
        
    BoxScore_df['GAME_ID'] = BoxScore_df['GAME_ID'].astype(int)
    BoxScore_df = pd.merge(BoxScore_df, game_dates, on='GAME_ID', how='left')
    BoxScore_df = BoxScore_df.reset_index().drop_duplicates(subset=['GAME_ID','PLAYER_ID'],keep='first')
    BoxScore_df['GAME_DATE'] =  pd.to_datetime(BoxScore_df['GAME_DATE'], errors='coerce')

    schedule_input = pd.read_excel('')[['Home_ID', 'Date']] # Define your own folder
    schedule_input['Last_HOME'] = 1
    BoxScore_df = pd.merge(BoxScore_df, schedule_input, left_on=['TEAM_ID', 'GAME_DATE'], right_on=['Home_ID', 'Date'], 
                           how='left').drop(['Home_ID', 'Date'], 1) 
    BoxScore_df['Last_HOME'] = BoxScore_df['Last_HOME'].replace(np.nan, 0, regex=True)
    return BoxScore_df


def advanced_boxscores(IDs, seconds): # Download advanced boxscores for chosen IDs
    pd.options.mode.chained_assignment = None
    Advanced_BoxScore_df = pd.DataFrame()
    for ID in tqdm.tqdm(IDs['GAME_ID']):
        time.sleep(seconds)
        BoxScore = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=ID)
        BoxScore = BoxScore.team_stats.get_data_frame()
        Advanced_BoxScore_df = Advanced_BoxScore_df.append(BoxScore)
        
    Advanced_BoxScore_df['GAME_ID'] = Advanced_BoxScore_df['GAME_ID'].astype(int)
    Advanced_BoxScore_df = pd.merge(Advanced_BoxScore_df, game_dates, on='GAME_ID', how='left')
    Advanced_BoxScore_df = Advanced_BoxScore_df.drop_duplicates()
    Advanced_BoxScore_df['GAME_DATE'] =  pd.to_datetime(Advanced_BoxScore_df['GAME_DATE'], errors='coerce')
    Advanced_BoxScore_df = Advanced_BoxScore_df.sort_values(by = ['TEAM_ID', 'GAME_DATE']) # sort values for rolling AVG   
    Advanced_BoxScore_df = Advanced_BoxScore_df.replace("", np.nan, regex=True) 
    Advanced_BoxScore_df.to_csv('advanced_boxscore_temp.csv')
    return Advanced_BoxScore_df


def missing_bscores():   
    games = get_games20162020() # run function defined earlier to get the list of all played games
    games = games.drop_duplicates('GAME_ID')
    gameids = games['GAME_ID']
    df1 = pd.DataFrame(data = gameids)
    df1.reset_index(drop=True)
    df1.index = pd.RangeIndex(len(df1.index))

    df2 = pd.read_sql_query("SELECT * FROM Appended_advanced", connection)['GAME_ID'].unique()
    df2 = pd.DataFrame(data = df2, columns = ['GAME_ID'])
    df2['GAME_ID'] = '00' + df2['GAME_ID'].astype(str)
    df3 = pd.concat([df1,df2])
    missing_advanced = df3.drop_duplicates(keep = False)

    df4 = pd.read_sql_query("SELECT * FROM Appended_basic", connection)['GAME_ID'].unique()
    df4 = pd.DataFrame(data = df4, columns = ['GAME_ID'])
    df4['GAME_ID'] = '00' + df4['GAME_ID'].astype(str)
    df5 = pd.concat([df1, df4])
    missing_basic = df5.drop_duplicates(keep = False)
    return missing_basic, missing_advanced


def update_bscores(missing_basic, missing_advanced):
    temp_basic_boxscores = basic_boxscores(missing_basic, 35)
    temp_basic_boxscores = temp_basic_boxscores.replace("", np.nan, regex=True)
    temp_basic_boxscores.to_sql('Appended_basic', connection, if_exists='append', index=False)
      
    temp_advanced_boxscores = advanced_boxscores(missing_advanced, 35)
    temp_advanced_boxscores = temp_advanced_boxscores.replace("", np.nan, regex=True)    
    temp_advanced_boxscores.to_sql('Appended_advanced', connection, if_exists='append')

    
def call_missing_bscores_update():
    # Look up missing boxscores of new games and insert them into the dataframe
    missing_basic, missing_advanced = missing_bscores()
    print ('missing basic bscores: {}'.format(missing_basic))
    if (len(missing_basic) > 0 or len(missing_advanced) > 0):
        update_bscores(missing_basic, missing_advanced)
    review_dictionary.update({'Added missing basic': len(missing_basic)})
    review_dictionary.update({'Added missing advanced': len(missing_basic)})
    print ('Updated {} missing basic and {} missing advanced boxscores'.format(len(missing_basic), len(missing_basic)))
    
    
def control_lineup_result():  
    # Calculate control lineup actual DF result
    control_lineup = pd.read_sql_query("SELECT * FROM Control_lineup", connection)
    basic_boxscore = pd.read_sql_query("SELECT * FROM Appended_basic", connection)
    basic_boxscore = basic_boxscore.sort_values('GAME_DATE').drop_duplicates('PLAYER_ID', keep='last')
    control_lineup_results = (
        pd.merge(control_lineup, basic_boxscore, how='left', left_on='PLAYER_ID', right_on='PLAYER_ID'))
    output_lineup_sum = control_lineup_results['DF_PTS'].sum()
    return output_lineup_sum


def review():
    review_dictionary.update({'Date': date_extension})
    review_dictionary_df = pd.DataFrame.from_dict(review_dictionary, orient='index')
    try:
        review_dictionary_df.to_sql('review_data', connection, if_exists = 'replace', index=False)
    except Exception:
        pass
    # df has to be in row when adding to sql, but column for sending the email
    review_dictionary_df = review_dictionary_df.transpose().round(1) 
    return review_dictionary_df


def new_season_cleanup(): # Tables that should be empty with the start of the new season
    user_input = input('Are you sure you want to clean all the SQL tables Y/N: ')
    if user_input == 'Y':
        db_tables = ['Appended_basic', 'Appended_advanced', 'review_data', 'Control_lineup', 'Combined_boxscores', 
                    'modelling_data', 'Prediction_calc_data', 'Predictions_data', 'Purchase_data', 'Fanteam_results',
                    'Player_positions', 'Predictions_df', 'Progress']
        c = connection.cursor() 
        for table in db_tables:
            c.execute("DELETE FROM {};".format(table))
        connection.commit()
    else:
        pass

## Data wrangling and preparation for learning part






In [15]:
def rolling_average_df(lag1, lag2, lag3): # Data preparation for modelling/learning
    BoxScore_basic_df = pd.read_sql_query("SELECT * FROM Appended_basic", connection)
    BoxScore_basic_df = BoxScore_basic_df.replace("", np.nan, regex=True)
    BoxScore_basic_df['GAME_DATE'] = pd.to_datetime(BoxScore_basic_df['GAME_DATE'], errors='coerce')
    BoxScore_basic_df = BoxScore_basic_df.sort_values(by = ['PLAYER_ID', 'GAME_DATE']) # sort values for rolling averages
    BoxScore_basic_df = BoxScore_basic_df.reset_index(drop=True)
    # Fill in some data that is missing
    BoxScore_basic_df = BoxScore_basic_df.groupby(['PLAYER_ID'], as_index=False).fillna(method = 'ffill', limit = 2)
    review_dictionary.update({'Total # of basic BS games:': BoxScore_basic_df['GAME_ID'].nunique()})

    # This is to be used not for learning, but predictions later (thus is included in the return, rolling not needed)
    actual_basic_df = BoxScore_basic_df.copy(deep=True)
    
    for stat in Constants.BASIC_STATS:
        BoxScore_basic_df['Last{lag}_{s}'.format(lag = lag1, s = stat)] = (
        BoxScore_basic_df.groupby(['PLAYER_ID'])[stat].apply(lambda x: rolling_average(x,lag1,1)))
        BoxScore_basic_df['Last{lag}_{s}'.format(lag = lag2, s = stat)] = (
        BoxScore_basic_df.groupby('PLAYER_ID')[stat].apply(lambda x: rolling_average(x,lag2,1)))
        BoxScore_basic_df['Last{lag}_{s}'.format(lag = lag3, s = stat)] = (
        BoxScore_basic_df.groupby('PLAYER_ID')[stat].apply(lambda x: rolling_average(x,lag3,1)))
       
    BoxScore_advanced_df = pd.read_sql_query("SELECT * FROM Appended_advanced", connection)
    BoxScore_advanced_df = BoxScore_advanced_df.replace("", np.nan, regex=True)
    BoxScore_advanced_df = BoxScore_advanced_df.drop_duplicates(subset=['GAME_ID','TEAM_ID'],keep='first')
    BoxScore_advanced_df['GAME_DATE'] = pd.to_datetime(BoxScore_advanced_df['GAME_DATE'], errors='coerce')
    BoxScore_advanced_df = BoxScore_advanced_df.sort_values(by = ['TEAM_ID', 'GAME_DATE'])
    BoxScore_advanced_df = BoxScore_advanced_df.reset_index(drop=True) 
    
    review_dictionary.update({'Total # of advvanced BS games:': BoxScore_advanced_df['GAME_ID'].nunique()})  
    # Fill in some data that is missing
    BoxScore_advanced_df = BoxScore_advanced_df.groupby(['TEAM_ID'], as_index=False).fillna(method = 'ffill', limit = 2)
    
    # This is to be used not for learning, but predictions later (thus is included in the return, rolling not needed)
    actual_advanced_df = BoxScore_advanced_df.copy()
    
    for stat in Constants.ADVANCED_STATS:
        BoxScore_advanced_df['Last{lag}_{s}'.format(lag = lag1, s = stat)] = (
        BoxScore_advanced_df.groupby('TEAM_ID')[stat].apply(lambda x: rolling_average(x,lag1,1)))
        BoxScore_advanced_df['Last{lag}_{s}'.format(lag = lag2, s = stat)] = (
        BoxScore_advanced_df.groupby('TEAM_ID')[stat].apply(lambda x: rolling_average(x,lag2,1)))
        BoxScore_advanced_df['Last{lag}_{s}'.format(lag = lag3, s = stat)] = (
        BoxScore_advanced_df.groupby('TEAM_ID')[stat].apply(lambda x: rolling_average(x,lag3,1)))

    # Switch game values that are relevant for opposition 
    BoxScore_advanced_df = BoxScore_advanced_df.sort_values(by = ['GAME_ID', 'TEAM_ID'])
    df2 = BoxScore_advanced_df[switching_columns_1].copy().reset_index(drop=True)
    df3 = BoxScore_advanced_df[switching_columns_1].copy().reset_index(drop=True)
    df2[switching_columns_2] = BoxScore_advanced_df.groupby('GAME_ID')[switching_columns_2].shift(-1)
    df3[switching_columns_2] = BoxScore_advanced_df.groupby('GAME_ID')[switching_columns_2].shift(1)
    for col in switching_columns_2:
        df2[col].fillna(df3[col], inplace=True) # This is to make a join between the two DFs with shifted values
    
    # In the main DF replace existing 'switching columns' with switched ones. 
    BoxScore_advanced_df[switching_columns_2] = df2[switching_columns_2]
    
    combined_boxscores = pd.merge(BoxScore_basic_df, BoxScore_advanced_df, on=['GAME_ID', 'TEAM_ID'], how='left')
    combined_boxscores = combined_boxscores.drop_duplicates(subset=['GAME_ID','PLAYER_ID'], keep='first')
    drop_y(combined_boxscores)
       
    
    # Add rest days to the combined boxscore dataframe
    combined_boxscores['GAME_DATE'] = pd.to_datetime(combined_boxscores['GAME_DATE'], errors='coerce') 
    combined_boxscores['Last_rest_days'] = combined_boxscores.groupby('PLAYER_ID')['GAME_DATE'].diff()
    combined_boxscores['Last_rest_days'] = combined_boxscores['Last_rest_days'] / np.timedelta64(1,'D')
    combined_boxscores['Last_rest_days'] = combined_boxscores['Last_rest_days'].replace(
        np.nan, 7, regex=True) # with no prior game,rest is capped at 7 days
    
    combined_boxscores.to_sql('Combined_boxscores', connection, if_exists='replace')
    combined_boxscores.to_csv('combined_boxscores_sanity.csv')
    
    # Define list of columns that will be attributes in learning
    learning_attributes = list(combined_boxscores.loc[:, combined_boxscores.columns.str.startswith('Last')].columns)
    data_model_df = combined_boxscores.dropna(subset=learning_attributes).reset_index(drop=True)
    data_model_df = data_model_df[learning_attributes]
    review_dictionary.update({'Total # of attributes': len(learning_attributes)})
    data_model_df.to_sql('modelling_data', connection, if_exists='replace')
    
    return data_model_df, combined_boxscores, actual_basic_df, actual_advanced_df, learning_attributes

## Create training/testing model

In [16]:
def modelling():
    connection_2018 = sqlite3.connect('') # Define your own folder
    data_model_df = pd.read_sql_query("SELECT * FROM modelling_data", connection_2018).iloc[:,2:]
    
    # Define type of estimator used in modelling
    estimators = ['RandomForestRegressor']
    types = ['train', 'test']
    coefficients = ['rmse', 'r2']
    
    # Create an empty table (estimators vs types)
    rmse_names = [x + '_' + y for y in types for x in estimators] 
    df_rmse = pd.DataFrame([[0.0] * 2 for j in range(len(rmse_names))], index = rmse_names, columns = coefficients)

    # Data preparation to put into scikit. Depended variable y and Independent variables, attributes, X
    y = data_model_df['DF_PTS']    
    X = data_model_df[learning_attributes].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(X)
    X = pd.DataFrame(x_scaled, columns=learning_attributes)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)    
    print ('Data shapes. X_train: {}, y_train: {}, X_test: {}, y_test: {}'.format(
        X_train.shape, y_train.shape, X_test.shape, y_test.shape))
    
    # Training the model. Currently only with Rand. Forest, but left flexibility for additional types
    for i in range(len(estimators)):
        est = estimators[i]
        if(est == 'RandomForestRegressor'):
            n_estimators = [100]
            param_grid = {'n_estimators': n_estimators}
            rf = GridSearchCV(RandomForestRegressor(max_depth=7), param_grid, cv=5)
            rf.fit(X_train, y_train) 
        
    #Calculating statistical significance
    train_rmse = np.sqrt(np.mean((y_train - rf.predict(X_train))**2.0 ))
    test_rmse = np.sqrt(np.mean((y_test - rf.predict(X_test))**2.0 ))
    train_r2  = r2_score(y_train, rf.predict(X_train))
    test_r2 = r2_score(y_test, rf.predict(X_test))

    # Adding above calculated measures into the created empty table
    for val in types:
        for coef in coefficients:
            df_rmse.loc[estimators[i] + '_' + val, coef] = eval(val + '_' + coef)  
    print ('Model performance: {}'.format(df_rmse))   
        
    # Save the trained model
    Nba_model_pickle = '' # Define your own folder
    pickle.dump(rf, open(Nba_model_pickle, 'wb'))

## Data wrangling and preparation for predictions part

In [17]:
def evaluation_preparation(lag1, lag2, lag3): # Prepare data for real valuation
    for stat in Constants.BASIC_STATS:
        actual_basic_df['Last{lag}_{s}'.format(lag = lag1, s = stat)] = (
        actual_basic_df.groupby('PLAYER_ID')[stat].apply(lambda x: rolling_average(x,lag1,0)))
        actual_basic_df['Last{lag}_{s}'.format(lag = lag2, s = stat)] = (
        actual_basic_df.groupby('PLAYER_ID')[stat].apply(lambda x: rolling_average(x,lag2,0)))
        actual_basic_df['Last{lag}_{s}'.format(lag = lag3, s = stat)] = (
        actual_basic_df.groupby('PLAYER_ID')[stat].apply(lambda x: rolling_average(x,lag3,0)))  
    
    for stat in Constants.ADVANCED_STATS:
        actual_advanced_df['Last{lag}_{s}'.format(lag = lag1, s = stat)] = (
        actual_advanced_df.groupby('TEAM_ID')[stat].apply(lambda x: rolling_average(x,lag1,0)))
        actual_advanced_df['Last{lag}_{s}'.format(lag = lag2, s = stat)] = (
        actual_advanced_df.groupby('TEAM_ID')[stat].apply(lambda x: rolling_average(x,lag2,0)))
        actual_advanced_df['Last{lag}_{s}'.format(lag = lag3, s = stat)] = (
        actual_advanced_df.groupby('TEAM_ID')[stat].apply(lambda x: rolling_average(x,lag3,0)))  
    
    # Take the tail (1) value of the list, as that is the only one needed for predict (contains all last info)
    actual_basic_tail_df = actual_basic_df.sort_values(by=['PLAYER_ID','GAME_DATE']).groupby('PLAYER_ID').tail(1)
    actual_advanced_tail_df = actual_advanced_df.sort_values(by=['TEAM_ID','GAME_DATE']).groupby('TEAM_ID').tail(1)  
        
    # Calculate how long ago was the last game played compared to today
    actual_basic_tail_df['GAME_DATE'] = pd.to_datetime(actual_basic_tail_df['GAME_DATE'], errors='coerce')
    today = pd.Timestamp("today").strftime("%m/%d/%Y")
    today = pd.to_datetime(today, errors='coerce')
    actual_basic_tail_df.loc[:,'Last_rest_days'] = (
        actual_basic_tail_df.loc[:,'GAME_DATE'] - datetime.datetime.strptime(date_extension, "%Y-%m-%d")) * -1      
    actual_basic_tail_df['Last_rest_days'] = actual_basic_tail_df['Last_rest_days'] / np.timedelta64(1,'D')
    actual_basic_tail_df.to_csv('actual_basic_tail_df_sanity.csv')
    
    review_dictionary.update({'Average rest time as of today': actual_basic_tail_df['Last_rest_days'].mean()})
    
    # Take input (scheduled todays' games) and merge them with the data on those teams respectively
    schedule_input = pd.read_excel('') # Define your own folder
    schedule_input = schedule_input.where(schedule_input['Date'] == date_extension).dropna()
    df1 = pd.merge(actual_basic_tail_df, schedule_input, how = 'left',
                   left_on = str('TEAM_ID'), right_on = str('Home_ID')).drop('Home_ID',1)
    df2 = pd.merge(actual_basic_tail_df, schedule_input, how = 'left',
                   left_on = str('TEAM_ID'), right_on = str('Away_ID')).drop('Away_ID',1)
    df1['Home_ID'] = df2['Home_ID']
    df1['Opponent'] = df1['Away_ID'].fillna(df1['Home_ID'])
    df3 = pd.merge(df1, actual_advanced_tail_df[switching_columns_1],
                   how= 'left', left_on = 'Opponent', right_on = 'TEAM_ID')
    df3[learning_attributes] = df3[learning_attributes].round(2)
    drop_y(df3)
    df3 = df3.dropna(subset = learning_attributes).reset_index(drop=True)
   
    # Data processing for model input, the same way as in learning part
    df4 = df3[learning_attributes].values
    min_max_scaler = preprocessing.MinMaxScaler()
    df4 = min_max_scaler.fit_transform(df4)
    df4 = pd.DataFrame(df4, columns = learning_attributes)    
    df4.to_sql('Prediction_calc_data', connection, if_exists='replace')
    
    # Call controll lineup performance calculation before the new control lineup is created
    review_dictionary.update({'Control_lineup_performance': control_lineup_result()}) 
    return df4, df3

## Betting-website data

In [ ]:
import Fanteam_data # Web scraping script that I will not share. Please reach out if you would like some advice on this

## Making predictions

In [18]:
def predict(df, fanteam_salaries, predicted_players):
    rf = pickle.load(open('', 'rb')) # Define your own folder
    predictions = rf.predict(df)
    
    # Convert predictions array to a dataframe
    ind = [str(i) for i in range(1, len(predictions)+1)] # make index for the dataframe being created
    predictions_df = pd.DataFrame(predictions, index = ind, columns = ['predictions']).reset_index()
    predictions_df['PLAYER_ID'] = predicted_players['PLAYER_ID']
    predictions_df.drop(columns = ['index'], inplace = True)
    predictions_df = pd.merge(predictions_df, fanteam_salaries, \
             how = 'left', left_on = 'PLAYER_ID', right_on = 'PERSON_ID').drop('PERSON_ID', 1) #Add column with positions
    
    predictions_df = predictions_df.rename(columns={'SALARIES': 'Salary'})
    predictions_df['Salary'] = predictions_df['Salary'].str.replace('M','') # Convert salary string to numbers
    predictions_df['Salary'] = pd.to_numeric(predictions_df['Salary'], errors='coerce')*1000000
    
    #predictions_df['Salary'] = np.random.randint(5000, 15000, predictions_df.shape[0])
    predictions_df['Date'] = date_extension
    # Adding a column in the predictions dataframe that will contain simple average to be used for control team.
    predictions_df = predictions_df.sort_values(by=['PLAYER_ID'])
    predicted_players = predicted_players.sort_values(by=['PLAYER_ID'])
    
    predictions_df = pd.merge(predictions_df, predicted_players[['PLAYER_ID', 'Last5_DF_PTS']], how='left', on='PLAYER_ID')
    predictions_df = predictions_df.rename({'Last5_DF_PTS': 'Control_last5'}, axis='columns') # to be changed back to15,5,1

    review_dictionary.update({'Total # of predicted players': predictions_df['PLAYER_ID'].shape[0]})
    return predictions_df

# Optimization process

In [19]:
def optimize_lineup(predictions_df, budget, players_limit, type='main'):    
    # Create a distinction between ML optimization and control/base optimization by selecting column
    if type == 'main':
        main_variable = 'predictions'
    elif type == 'control':
        main_variable = 'Control_last5'
    else:
        raise ValueError('Value entered is not for main or control optimization')
    
    predictions_df = predictions_df.dropna(subset=['Salary', 'PLAYER_NAMES', 'Control_last5']).reset_index(drop=True)
    print ('The number of players used in optimization is: {}'.format(len(predictions_df['Salary'])))
    
    # Initialize pulp with a optimization objective
    problem = pulp.LpProblem('Maximization problem', pulp.LpMaximize)

    # Create binary variable (column) for each of the players in the dataset
    decision_variables = []
    for rownum, row in predictions_df.iterrows():
        variable = str('x' + str(rownum))
        variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer')
        decision_variables.append(variable)    

    # Create objective function - sum of predicted DF points multiplied by 0/1 - binary player variable
    Sum_predictions = ""
    for rownum, row in predictions_df.iterrows():
        for i, binary in enumerate(decision_variables):
            if (rownum == i):
                Sum_predictions += row[main_variable] * binary
    problem += Sum_predictions

    # Implementing all constraints
    number_of_players = ""
    total_salary = ""
    number_of_pg = ""
    number_of_sg = ""
    number_of_sf = ""
    number_of_pf = ""
    number_of_c  = ""

    for rownum, row in predictions_df.iterrows():
        for i, binary in enumerate(decision_variables):
            if (rownum == i):
                total_salary += row['Salary'] * binary
                number_of_players += binary
                if row['PLAYER_POSITIONS'] == 'PG':
                    number_of_pg += binary * 1               
                if row['PLAYER_POSITIONS'] == 'SG':
                    number_of_sg += binary * 1
                if row['PLAYER_POSITIONS'] == 'SF':
                    number_of_sf += binary * 1    
                if row['PLAYER_POSITIONS'] == 'PF':
                    number_of_pf += binary * 1    
                if row['PLAYER_POSITIONS'] == 'C':
                    number_of_c += binary * 1         

    # Constraint limits
    problem += (total_salary <= budget)
    problem += (number_of_players == Constants.POSITIONAL_LIMITS['TOTAL'])
    problem += (number_of_pg <= Constants.POSITIONAL_LIMITS['PG'])
    problem += (number_of_sg <= Constants.POSITIONAL_LIMITS['SG'])
    problem += (number_of_sf <= Constants.POSITIONAL_LIMITS['SF'])
    problem += (number_of_pf <= Constants.POSITIONAL_LIMITS['PF'])
    problem += (number_of_c  <= Constants.POSITIONAL_LIMITS['C'])
    problem += (1 <= number_of_pg)
    problem += (1 <= number_of_sg)
    problem += (1 <= number_of_sf)
    problem += (1 <= number_of_pf)
    problem += (1 <= number_of_c)
    
    
    
    
    # Run optimization
    optimization_result = problem.solve()
    
    # Present results, using optimization results to create a dataframe with that data
    variable_name = []
    variable_value = []
    for v in problem.variables():
        variable_name.append(v.name)
        variable_value.append(v.varValue)
    output = pd.DataFrame({'Variable': variable_name, 'Value': variable_value})

    for rownum, row in output.iterrows():
        value = re.findall(r'(\d+)', row['Variable'])
        output.loc[rownum, 'variable'] = int(value[0])
        output = output.sort_values(by='variable').reset_index(drop = True)
        
    # Attaching the output column to the dataframe with original statistics information
    for rownum, row in predictions_df.iterrows():
        for results_rownum, results_row in output.iterrows():
            if (rownum == results_rownum):
                predictions_df.loc[rownum, 'Purchase_decision'] = results_row['Value']    

    # Attach player names to the decisions and filter out only the purchasing ones                
    predictions_df = (pd.merge(predictions_df, players[['PERSON_ID', 'PLAYERCODE']],
                    how = 'left', left_on = 'PLAYER_ID', right_on = 'PERSON_ID'))  
    predictions_df['Date'] = date_extension
    purchase_df = predictions_df.loc[predictions_df['Purchase_decision'] == 1]
        
    if type == 'main':
        predictions_df.to_sql('Predictions_data', connection, if_exists='replace')
        purchase_df.to_sql('Purchase_data', connection, if_exists='replace')
        review_dictionary.update({'Total # of variales (optimization)': len(decision_variables)})
        review_dictionary.update({'Total # of missing salaries': predictions_df['Salary'].isna().sum()})
        review_dictionary.update({'Total DF salary': purchase_df['Salary'].sum()})
        review_dictionary.update({'Total predicted DF points': purchase_df['predictions'].sum()})
    elif type == 'control':
        review_dictionary.update({'Total base DF salary': purchase_df['Salary'].sum()})
        review_dictionary.update({'Total base predicted DF points': purchase_df['Control_last5'].sum()})
        purchase_df['PLAYER_ID'].to_sql('Control_lineup', connection, if_exists='replace')
           
    print ('{} lineup is ready!'.format('Optimal' if type=='main' else 'Control'))
    return purchase_df

## Running main module

In [ ]:
 new_season_cleanup() 

# Start a separate thread to run and fetch fanteam data in parallel with getting boxscore data
fanteam_returns = dict() # empty dictionary to collect the results (scraped data) of imported fanteam_data function 
fanteam_thread = threading.Thread(target=Fanteam_data.fanteam_data, args=(fanteam_returns,))
fanteam_thread.start() # Thread starts to scrape fanteam data, following rows executed in paralel

game_dates = get_game_dates()

players = player_list()

switching_columns_1, switching_columns_2 = switching_columns(lag1, lag2, lag3)

call_missing_bscores_update()

data_model_df, data_df, actual_basic_df, actual_advanced_df, learning_attributes = rolling_average_df(lag1, lag2, lag3) # +

if raw_input('Would you like to run modelling 1/0?') == 1: modelling()

prediction_ready_data, predicted_players = evaluation_preparation(lag1, lag2, lag3) 

fanteam_thread.join() # join makes the main thread stop and wait for fanteam_thread to finish before continuing main

predictions_df = predict(prediction_ready_data, fanteam_returns['salaries_df'], predicted_players)

optimal_lineup = optimize_lineup(predictions_df,fanteam_returns['budget'],fanteam_returns['players_limit'], type='main')
control_lineup = optimize_lineup(predictions_df,fanteam_returns['budget'],fanteam_returns['players_limit'], type='control')

review_dictionary_df = review()

Send_email.send_email(
    optimal_lineup[['PLAYER_POSITIONS', 'PLAYERCODE', 'Salary']],
    control_lineup[['PLAYER_POSITIONS', 'PLAYERCODE', 'Salary']],
    review_dictionary_df)